In [1]:
import requests
import matplotlib.pyplot as plt
from IPython.display import Image
import folium
%matplotlib inline

# APIs

An Application Programming Interface, or API, is a structured way to retrieve data from a website. Using an API is safer and easier than something like webscraping, since what you get back is already in a usable format. Many organizations use APIs like:
- Government organizations ([US Government](https://www.data.gov/developers/apis))
- Large companies ([Twitter API](https://developer.twitter.com/en/docs))
- News organizations ([NYT API](https://developer.nytimes.com/))
- And [many more](https://github.com/public-apis/public-apis)

If you type `how to use an api in python` in google, you get back many articles walking through how to use an API. It is a well documented and useful tool to be familiar with.

### API Keys

One of the main ways APIs maintain security is by the use of some form of authentication, such as an API key. An API key can be obtained in a number of ways, depending on the API, and is a way for the application to know who you are and provides you secure access to the data.

**DO NOT SHARE YOUR API KEYS OR PUT THEM IN A PUBLIC PLACE LIKE GITHUB**

API keys should be stored securely on your computer and removed from any code or documents you share.

#### Lets use the NASA API as an initial example: https://api.nasa.gov/

1. Scroll down and enter your First Name, Last Name, and email to generate an API key
2. Put the API key in a safe place like a text file that won't be shared
3. Copy the API key into the `api_key` variable in the cell below as a string

In [ ]:
api_key =

### Endpoints

We will be using the `requests` package to make a `GET` request to an API. Similar to webscraping, APIs require an endpoint to tell python where to send the request.
1. Look at the different available APIs in the `Browse APIs` tab
2. Click on the InSight: Mars Weather Service API option
3. Look at the `Query Parameters` and compare that to what is under the `HTTP Request` header
4. Copy what is under the `HTTP Request` header into the endpoint variable below as a string

In [ ]:
endpoint =

The `GET` part of the endpoint is not needed, as that is the request type and we will specify that in another way
- delete `GET`

There is a typo in the endpoint
- delete the space after `https:`

Everything after the `?` are parameters that specify what information to pull back, we will specify this in a different way
- delete the `?` and everything after it

### Parameters

Parameters are specific to each API and indicate what information you want back. These can be compared to the various ways you slice a table or df to get just the subset you want. Some parameters are required, others are optional. Always look at the documentation to know what parameters you should include and what are possible values for each one.

When using the `requests` package, parameters are specified in a **dictionary**.
1. Make an empty dictionary for the `params` variable below
2. Look at the documentation to know what parameters you should include, add these as **keys** to the dictionary
3. Add the appropriate values for each parameter as the **values** for the dictionary

In [ ]:
params = 

You now have all the pieces to make an API request

In [ ]:
response = requests.get(endpoint, params = params)

See what was saved to `response`

In [ ]:
response

Add a `.` and press tab to see what possible methods you can perform on `response`. Choose one that seems like it would be useful to pull out the information you're looking for. After some experimentation, put your best option in the cell below and save the output to `res`

In [ ]:
res = response.json()

In [ ]:
res

Based on some other outputs, the fields represent tempreture (AT), winds (HWS), pressure (PRE), as well as more detailed information about winds. The documentation also indicates that we will get data for the last 7 Sols on Mars. We will use this information to pull out the average temperature for each of the 7 days and visualize the change over time.

In [ ]:
temps = []
for day_num, day_info in res.items():
    try:
        temps.append(day_info['AT']['av'])
    except (TypeError, KeyError):
        print(day_num)

In [ ]:
plt.plot(temps);

---

Let's try another `endpoint` from NASA. This time copy the endpoint from the **Earth** section

In [ ]:
endpoint = 

Add the appropriate parameters (Use the city center of Nashville as a starting point: (36.186314, -87.0654301)). Be sure to look at which parameters have defaults and which do not (those are required parameters)

In [ ]:
params = {}

In [ ]:
response = requests.get(endpoint, params = params)

We'll now use the `.content` attribute from the response to render an image

In [ ]:
Image(response.content)

---

For the last example we'll switch to using one of the APIs from Google Maps, the `nearbysearch`

For now just follow along, but if you want to test this out in the future, you can follow these steps to get your API key (https://developers.google.com/maps/documentation/embed/get-api-key):
1. Create an account on the Google Cloud Platform Console: console.cloud.google.com
2. Create a new project
3. Select the newly created project, open the drop down menu on the left, and select APIs & Services > Credentials
4. Click Create credentials > API key
5. Your new API key will appear in the text box, copy it to a safe place
6. Click Close

In [ ]:
def find_nearby_places(api_key,
                      location,
                      radius,
                      base = 'https://maps.googleapis.com/maps/api/place/nearbysearch/',
                      response_type = 'json',
                      **kwargs):
    '''
    Constructs endpoint and params dict, makes request, and returns output as json
    
    api_key: requred, str, API key from google cloud console
    location: required, str, central location for near by search, format: "<lat>, <lng>"
    radius: required, str, radius of circle (in meters) to search for nearby locations
    base: not required, str, URL base for API request
    resoponse_type: not required, format of response
    **kwargs: not required, search criteria for near by locations
    '''
    endpoint = base+response_type
    kwargs['key'] = api_key
    kwargs['location'] = location
    kwargs['radius'] = radius
    res = requests.get(endpoint, kwargs)
    return res.json()

In [ ]:
response = find_nearby_places()

In [ ]:
response

In [ ]:
nash_map = folium.Map([36.1612, -86.7775], zoom_start = 12)

for store in response['results']:
    lat_lng = store['geometry']['location']
    location = [lat_lng['lat'], lat_lng['lng']]
    popup = store['name']
    folium.Marker(location = location, popup = popup).add_to(nash_map)

nash_map